In [ ]:
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import numbers, PatternFill
import os
from datetime import datetime
import numpy as np

from config_parameters import (
    ANALYSIS_TICKER_INTERVAL_FOLDER,
    SUMMARY_SIMULATIONS_FILE,
    ANALYSIS_STRATEGIES_FILE1,
    SIMULATION_DATA_DICT
)

# Define fill colors for positive and standout values
light_green = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
dark_green = PatternFill(start_color="006100", end_color="006100", fill_type="solid")

period_id = '2'
categories = ['CRYPTO', 'CRYPTO NEW']

start_date = "2025-01-01"
end_date = "2025-03-01"

sheet_output = os.path.join(SUMMARY_SIMULATIONS_FILE, f"{start_date}_to_{end_date}.xlsx")

# Define used intervals
intervals = ['1w', '3d', '2d', '1d', '12h', '6h', '4h', '2h', '1h', '30m', '15m']

# Load summary dataframe
summary_df = pd.read_excel(sheet_output)

# Filter data by selected categories
summary_df = summary_df[summary_df['Category'].isin(categories)]

# Unique tickers and intervals
unique_tickers = summary_df['Ticker'].unique()
unique_intervals = summary_df['Interval'].unique()

# --- Grouping data ---
grouped = summary_df.groupby(['Strategy', 'Interval']).agg(
    Count=('P&L after fees', 'count'),
    Sum_Deposit=('Deposit', 'sum'),
    Sum_PnL=('P&L after fees', 'sum'),
    Sum_Number=('Number', 'sum')
).reset_index()

grouped['PnL_per_Deposit'] = grouped['Sum_PnL'] / grouped['Sum_Deposit']

# Create pivot table for P&L after fees
pivot_df = pd.pivot_table(
    summary_df,
    values='P&L after fees',
    index='Strategy',
    columns='Interval',
    aggfunc='sum',
    fill_value=0,
    margins=True,
    margins_name='Total'
)

# Reorder columns based on defined interval list
desired_order = intervals + ['Total'] if 'Total' in pivot_df.columns else intervals
existing_columns = [col for col in desired_order if col in pivot_df.columns]
pivot_df = pivot_df[existing_columns]

# Prepare Excel Workbook
sheet_name = f"{start_date} - {end_date}"
wb = Workbook()
ws = wb.active
ws.title = sheet_name

# Add headers
header = f"Start date: {start_date} | End date: {end_date}"
ws.append([header])

deposit_value = summary_df['Deposit'].iloc[0]
deposit_per_interval = deposit_value * len(unique_tickers)
total_strategy_deposit = deposit_per_interval * len(unique_intervals)

header = f"Categories: {', '.join(categories)} | Ticker count: {len(unique_tickers)} | Strategy deposit: {total_strategy_deposit} | Interval deposit: {deposit_per_interval}"
ws.append([header])

ws.append(["P&L after fees summary"])

# Round and convert pivot to integers
pivot_df_rounded = pivot_df.round(0).astype(int)
pivot_data = pivot_df_rounded.reset_index()
first_strategy = pivot_df_rounded.index[0]

# Insert pivot table into sheet
for i, row in enumerate(dataframe_to_rows(pivot_data, index=False, header=True)):
    ws.append(row)
    if i == 0:
        continue

    current_row = ws.max_row
    strategy_name = row[0]

    for j, cell in enumerate(ws[current_row][1:], start=1):  # Skip 'Strategy' column
        value = cell.value
        interval = pivot_df_rounded.columns[j - 1]
        if isinstance(value, (int, float)):
            if value > 0:
                cell.fill = light_green
                reference_value = pivot_df_rounded.loc[first_strategy, interval]
                if strategy_name != first_strategy and value > reference_value:
                    cell.fill = dark_green

# Save to Excel file
excel_path = ANALYSIS_STRATEGIES_FILE1

# === Sheet handling ===
if os.path.exists(excel_path):
    wb_existing = load_workbook(excel_path)

    # Remove existing sheet with same name
    if sheet_name in wb_existing.sheetnames:
        std = wb_existing[sheet_name]
        wb_existing.remove(std)

    # Add sheets from the original file to the new workbook
    for sheet in wb_existing.worksheets:
        if sheet.title not in wb.sheetnames:
            wb._add_sheet(sheet)

# Ensure the current sheet exists
if sheet_name not in wb.sheetnames:
    wb.create_sheet(title=sheet_name)

wb.save(excel_path)
print(f"✅ Workbook successfully saved to: {excel_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'Simulations_summaries\\2025-01-01_to_2025-03-01.xlsx'